 ## 1. ライブラリのインポート

 必要なライブラリをインポートします。

In [1]:
import pybotters
import pandas as pd
import asyncio
import time
from datetime import datetime, timedelta, timezone


 ## 2. 設定値の定義

 データ取得に必要なパラメータを設定します。

 - `symbol`: Bybitでの通貨ペアシンボル (ここでは流動性の高いBTCUSDTを例とします)

 - `interval`: 時間足 ('5' = 5分足)

 - `category`: Bybit V5 APIのカテゴリ ('linear' = USDTまたはUSDC無期限契約)

 - `limit`: 1回のリクエストで取得する最大件数 (Bybit V5では最大1000)

 - `num_requests`: 取得するリクエスト回数 (例: 1000件 * 10回 = 10000件)

 - `filename`: 取得したデータを保存するファイル名

In [2]:
# --- 設定項目 ---
target_symbol = 'BTCUSDT' # Bybitのシンボル名 (例: BTCUSDT, ETHUSDTなど)
interval = '5'          # 時間足 ('1', '3', '5', '15', '30', '60', '120', ...)
category = 'linear'     # APIカテゴリ ('linear' or 'inverse' or 'spot')
limit = 1000            # 1回あたりの取得件数 (最大1000)
num_requests = 5        # 取得するリクエスト回数 (limit * num_requests 分のデータを取得試行)
output_filename = f'{target_symbol}_{interval}m_data.csv' # 保存ファイル名
# --- 設定項目ここまで ---

# Bybit APIのベースURL
base_url = 'https://api.bybit.com'


 ## 3. データ取得・整形関数の定義

 Bybit APIからデータを取得し、DataFrameに整形する非同期関数を定義します。

 Bybit V5 Kline APIは新しいデータから古いデータへ降順で返してくるため、複数回リクエストする場合はendTimeをずらしていきます。

In [3]:
async def fetch_bybit_kline(symbol, interval, category, limit, num_req):
    """Bybit V5 APIから複数回に分けてローソク足データを取得し、DataFrameに整形する"""
    apis = {}  # 公開データ取得なのでAPIキーは不要
    client = pybotters.Client(apis=apis, base_url=base_url)
    endpoint = '/v5/market/kline'

    all_data_list = []
    current_end_time = int(time.time() * 1000) # 現在時刻を起点とする (ミリ秒)

    print(f"データ取得を開始します (最大 {limit * num_req} 件)...")

    for i in range(num_req):
        params = {
            'category': category,
            'symbol': symbol,
            'interval': interval,
            'limit': limit,
            'end': current_end_time, # この時刻より前のデータを取得
        }
        try:
            resp = await client.get(endpoint, params=params)
            data = await resp.json()

            if data['retCode'] == 0 and data['result'] and data['result']['list']:
                kline_list = data['result']['list']
                print(f"{i+1}/{num_req}: {len(kline_list)} 件のデータを取得しました。 (終了時刻: {datetime.fromtimestamp(current_end_time / 1000, tz=timezone.utc).strftime('%Y-%m-%d %H:%M:%S')})")
                all_data_list.extend(kline_list)

                # 次のリクエストのために、取得した最も古いデータのタイムスタンプを次の終了時刻とする
                oldest_timestamp = int(kline_list[-1][0])
                # 同じデータを重複して取得しないように、1ミリ秒引く
                current_end_time = oldest_timestamp - 1

                # APIレートリミットを考慮して少し待機 (Bybit V5は 10 req/sec)
                await asyncio.sleep(0.2)

            else:
                print(f"{i+1}/{num_req}: データ取得エラーまたはデータがありません。Response: {data}")
                break # エラーまたはデータがなくなったら終了

        except Exception as e:
            print(f"{i+1}/{num_req}: リクエスト中にエラーが発生しました: {e}")
            # エラー発生時も少し待機
            await asyncio.sleep(1)
            continue # 次のリクエストを試行する場合

    if not all_data_list:
        print("有効なデータを取得できませんでした。")
        return pd.DataFrame()

    # DataFrameに変換
    df = pd.DataFrame(all_data_list, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'turnover'])

    # データ型の変換と整形
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True) # UTCとして読み込み
    df.set_index('timestamp', inplace=True)
    numeric_cols = ['open', 'high', 'low', 'close', 'volume', 'turnover']
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # 重複データの削除 (タイムスタンプ基準) とソート
    df = df[~df.index.duplicated(keep='first')]
    df.sort_index(ascending=True, inplace=True) # 昇順 (古い -> 新しい) に並び替え

    print(f"\nデータ取得完了。合計 {len(df)} 件の一意なデータを取得しました。")
    return df


 ## 4. データ取得の実行と結果確認

 上で定義した関数を実行してデータを取得し、DataFrameの基本情報を表示します。

 Jupyter Notebook等で実行する場合、nest_asyncioが必要になることがあります。

In [4]:
# --- nest_asyncioの適用 ---
import nest_asyncio
nest_asyncio.apply() # asyncio.run() をネスト可能にする

# --- 非同期処理の実行 ---
if __name__ == '__main__':
    try:
        print("データ取得処理を開始します...")
        df_kline = asyncio.run(fetch_bybit_kline(target_symbol, interval, category, limit, num_requests))
        print("データ取得処理が完了しました。")
    except Exception as e:
        print(f"データ取得中に予期せぬエラーが発生しました: {e}")
        df_kline = pd.DataFrame() # エラー時は空のDataFrameをセット

# --- 結果の表示 ---
if not df_kline.empty:
    print("\n--- 取得データ (最初の5行) ---")
    display(df_kline.head())

    print("\n--- 取得データ (最後の5行) ---")
    display(df_kline.tail())

    print("\n--- データフレーム情報 ---")
    df_kline.info()

    print(f"\n取得期間: {df_kline.index.min()} ~ {df_kline.index.max()}")
    print(f"データ件数: {len(df_kline)}")
else:
    print("\nデータフレームが空、または取得に失敗しました。")



データ取得処理を開始します...
データ取得を開始します (最大 5000 件)...
1/5: 1000 件のデータを取得しました。 (終了時刻: 2025-04-20 05:36:17)
2/5: 1000 件のデータを取得しました。 (終了時刻: 2025-04-16 18:19:59)
3/5: 1000 件のデータを取得しました。 (終了時刻: 2025-04-13 06:59:59)
4/5: 1000 件のデータを取得しました。 (終了時刻: 2025-04-09 19:39:59)
5/5: 1000 件のデータを取得しました。 (終了時刻: 2025-04-06 08:19:59)


C:\Users\trow1\AppData\Local\Temp\ipykernel_16504\784542691.py:55: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms', utc=True) # UTCとして読み込み
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x0000013E668F4440>
Unclosed connector
connections: ['deque([(<aiohttp.client_proto.ResponseHandler object at 0x0000013E64D58950>, 903937.9049848)])']
connector: <aiohttp.connector.TCPConnector object at 0x0000013E668F4590>



データ取得完了。合計 5000 件の一意なデータを取得しました。
データ取得処理が完了しました。

--- 取得データ (最初の5行) ---


,open,high,low,close,volume,turnover
timestamp,,,,,,
2025-04-02 21:00:00+00:00,85622.4,85698.4,85249.5,85278.7,613.816,5.243372e+07
2025-04-02 21:05:00+00:00,85278.7,85278.7,84649.6,84682.8,2867.092,2.434095e+08
2025-04-02 21:10:00+00:00,84682.8,84750.0,84425.0,84493.1,1754.793,1.483791e+08
2025-04-02 21:15:00+00:00,84493.1,84809.9,84168.0,84386.2,2073.124,1.751078e+08
2025-04-02 21:20:00+00:00,84386.2,84882.3,84292.7,84665.0,936.929,7.924680e+07



--- 取得データ (最後の5行) ---


,open,high,low,close,volume,turnover
timestamp,,,,,,
2025-04-20 05:15:00+00:00,85134.0,85143.1,85113.7,85126.9,22.834,1.943710e+06
2025-04-20 05:20:00+00:00,85126.9,85140.0,85118.7,85139.9,17.179,1.462333e+06
2025-04-20 05:25:00+00:00,85139.9,85140.0,85116.0,85116.0,16.444,1.399749e+06
2025-04-20 05:30:00+00:00,85116.0,85116.1,85085.0,85101.6,32.151,2.735974e+06
2025-04-20 05:35:00+00:00,85101.6,85101.6,85085.0,85091.1,7.525,6.403026e+05



--- データフレーム情報 ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5000 entries, 2025-04-02 21:00:00+00:00 to 2025-04-20 05:35:00+00:00
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   open      5000 non-null   float64
 1   high      5000 non-null   float64
 2   low       5000 non-null   float64
 3   close     5000 non-null   float64
 4   volume    5000 non-null   float64
 5   turnover  5000 non-null   float64
dtypes: float64(6)
memory usage: 273.4 KB

取得期間: 2025-04-02 21:00:00+00:00 ~ 2025-04-20 05:35:00+00:00
データ件数: 5000


 ## 5. (任意) データの保存

 取得したデータをCSVファイルとして保存します。

In [5]:
if not df_kline.empty:
    try:
        df_kline.to_csv(output_filename)
        print(f"\nデータを '{output_filename}' として保存しました。")
    except Exception as e:
        print(f"\nデータの保存中にエラーが発生しました: {e}")


データを 'BTCUSDT_5m_data.csv' として保存しました。


## 6. 特徴量の計算

取得したローソク足データから、AIモデルの入力となる特徴量を計算します。
ここでは、ドキュメントにあった以下の特徴量を計算します。

1. ローソク足の各値（始値, 高値, 安値）を終値で割った値（一種の正規化）
2. SMA (単純移動平均線)
3. RSI (相対力指数)

`pandas-ta`ライブラリを使用します。

In [6]:
import pandas_ta as ta

# df_klineが前のステップで取得したDataFrameとします
if 'df_kline' in locals() and not df_kline.empty:
    # 特徴量計算の準備 (コピーを作成して元のデータを保持)
    df_features = df_kline.copy()

    # 1. 終値による正規化
    df_features['open_norm'] = df_features['open'] / df_features['close']
    df_features['high_norm'] = df_features['high'] / df_features['close']
    df_features['low_norm'] = df_features['low'] / df_features['close']
    # 終値自身を終値で割ると常に1なので、特徴量としては通常含めませんが、
    # 陽線/陰線の区別などに使える場合もあるため、ここでは一旦 close_norm=1 も追加してみます。
    df_features['close_norm'] = 1.0 # df_features['close'] / df_features['close']

    # 2. SMA (例: 期間14)
    sma_period = 14
    df_features.ta.sma(length=sma_period, append=True) # 'SMA_14' という列が追加される

    # 3. RSI (例: 期間14)
    rsi_period = 14
    df_features.ta.rsi(length=rsi_period, append=True) # 'RSI_14' という列が追加される

    # 確認のため最初の数行を表示
    print("--- 特徴量計算後 (最初の数行) ---")
    display(df_features.head())

    print("\n--- データフレーム情報 (特徴量追加後) ---")
    df_features.info()

else:
    print("df_kline が存在しないか空です。データ取得ステップを先に実行してください。")



--- 特徴量計算後 (最初の数行) ---


,open,high,low,close,volume,turnover,open_norm,high_norm,low_norm,close_norm,SMA_14,RSI_14
timestamp,,,,,,,,,,,,
2025-04-02 21:00:00+00:00,85622.4,85698.4,85249.5,85278.7,613.816,5.243372e+07,1.004030,1.004922,0.999658,1.0,NaN,NaN
2025-04-02 21:05:00+00:00,85278.7,85278.7,84649.6,84682.8,2867.092,2.434095e+08,1.007037,1.007037,0.999608,1.0,NaN,NaN
2025-04-02 21:10:00+00:00,84682.8,84750.0,84425.0,84493.1,1754.793,1.483791e+08,1.002245,1.003040,0.999194,1.0,NaN,NaN
2025-04-02 21:15:00+00:00,84493.1,84809.9,84168.0,84386.2,2073.124,1.751078e+08,1.001267,1.005021,0.997414,1.0,NaN,NaN
2025-04-02 21:20:00+00:00,84386.2,84882.3,84292.7,84665.0,936.929,7.924680e+07,0.996707,1.002567,0.995603,1.0,NaN,NaN



--- データフレーム情報 (特徴量追加後) ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5000 entries, 2025-04-02 21:00:00+00:00 to 2025-04-20 05:35:00+00:00
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   open        5000 non-null   float64
 1   high        5000 non-null   float64
 2   low         5000 non-null   float64
 3   close       5000 non-null   float64
 4   volume      5000 non-null   float64
 5   turnover    5000 non-null   float64
 6   open_norm   5000 non-null   float64
 7   high_norm   5000 non-null   float64
 8   low_norm    5000 non-null   float64
 9   close_norm  5000 non-null   float64
 10  SMA_14      4987 non-null   float64
 11  RSI_14      4986 non-null   float64
dtypes: float64(12)
memory usage: 507.8 KB


## 7. 目的変数（ターゲット）の作成

AIに予測させたい目標、「5分後（次の足）の終値が現在の終値より高いか低いか」を作成します。
- 次の足の終値 > 現在の終値 なら 1 (High)
- 次の足の終値 <= 現在の終値 なら 0 (Low)

Pandasの `shift(-1)` を使うと、1行未来のデータを取得できます。

## 8. 不要な行（NaNを含む行）の削除

特徴量計算（SMA, RSIの初期期間）や目的変数作成（最後の行）によって生じた `NaN` を含む行は、AIモデルの学習には使えないため削除します。


In [7]:
if 'df_features' in locals() and not df_features.empty:
    # 次の足の終値を取得 (shift(-1)は1行未来のデータを取得)
    df_features['next_close'] = df_features['close'].shift(-1)

    # 目的変数の作成
    # np.where を使うと条件分岐で値を設定できる (import numpy as np が必要)
    import numpy as np
    df_features['target'] = np.where(df_features['next_close'] > df_features['close'], 1, 0)

    # 未来のデータがない最後の行は next_close が NaN になるため、target も不要
    # 不要になった next_close 列は削除しても良い
    # df_features = df_features.drop(columns=['next_close'])

    # 確認のため最後の数行を表示 (targetが計算されているか、最後の行はNaNか確認)
    print("--- 目的変数作成後 (最後の数行) ---")
    display(df_features.tail())

    # target列の構成を確認 (High=1, Low=0 の割合)
    print("\n--- 目的変数 (target) の分布 ---")
    print(df_features['target'].value_counts(dropna=False)) # dropna=FalseでNaNの数も表示

else:
    print("df_features が存在しないか空です。")


--- 目的変数作成後 (最後の数行) ---


,open,high,low,close,volume,turnover,open_norm,high_norm,low_norm,close_norm,SMA_14,RSI_14,next_close,target
timestamp,,,,,,,,,,,,,,
2025-04-20 05:15:00+00:00,85134.0,85143.1,85113.7,85126.9,22.834,1.943710e+06,1.000083,1.000190,0.999845,1.0,85148.628571,45.387608,85139.9,1
2025-04-20 05:20:00+00:00,85126.9,85140.0,85118.7,85139.9,17.179,1.462333e+06,0.999847,1.000001,0.999751,1.0,85150.378571,48.335835,85116.0,0
2025-04-20 05:25:00+00:00,85139.9,85140.0,85116.0,85116.0,16.444,1.399749e+06,1.000281,1.000282,1.000000,1.0,85151.614286,43.668421,85101.6,0
2025-04-20 05:30:00+00:00,85116.0,85116.1,85085.0,85101.6,32.151,2.735974e+06,1.000169,1.000170,0.999805,1.0,85148.100000,41.093692,85091.1,0
2025-04-20 05:35:00+00:00,85101.6,85101.6,85085.0,85091.1,7.525,6.403026e+05,1.000123,1.000123,0.999928,1.0,85142.978571,39.275269,NaN,0



--- 目的変数 (target) の分布 ---
target
0    2514
1    2486
Name: count, dtype: int64


In [8]:
if 'df_features' in locals() and not df_features.empty:
    # NaNを含む行を削除する前の行数を確認
    rows_before_dropna = len(df_features)
    print(f"NaN削除前の行数: {rows_before_dropna}")

    # NaNを含む行を削除
    df_processed = df_features.dropna()

    # NaNを含む行を削除した後の行数を確認
    rows_after_dropna = len(df_processed)
    print(f"NaN削除後の行数: {rows_after_dropna}")
    print(f"削除された行数: {rows_before_dropna - rows_after_dropna}")

    # 不要になった next_close 列を削除 (まだ残っていれば)
    if 'next_close' in df_processed.columns:
       df_processed = df_processed.drop(columns=['next_close'])


    # 最終的なデータフレームの情報を確認
    print("\n--- 最終的なデータフレーム情報 (NaN削除後) ---")
    df_processed.info()

    print("\n--- 最終的なデータフレーム (最初の5行) ---")
    display(df_processed.head())

    # この df_processed を次の機械学習ステップで使用します。
    # (任意) 必要であれば、この処理済みデータをCSV等で保存
    # output_processed_filename = f'{target_symbol}_{interval}m_processed.csv'
    # df_processed.to_csv(output_processed_filename)
    # print(f"\n処理済みデータを '{output_processed_filename}' として保存しました。")

else:
    print("df_features が存在しないか空です。")


NaN削除前の行数: 5000
NaN削除後の行数: 4985
削除された行数: 15

--- 最終的なデータフレーム情報 (NaN削除後) ---
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4985 entries, 2025-04-02 22:10:00+00:00 to 2025-04-20 05:30:00+00:00
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   open        4985 non-null   float64
 1   high        4985 non-null   float64
 2   low         4985 non-null   float64
 3   close       4985 non-null   float64
 4   volume      4985 non-null   float64
 5   turnover    4985 non-null   float64
 6   open_norm   4985 non-null   float64
 7   high_norm   4985 non-null   float64
 8   low_norm    4985 non-null   float64
 9   close_norm  4985 non-null   float64
 10  SMA_14      4985 non-null   float64
 11  RSI_14      4985 non-null   float64
 12  target      4985 non-null   int64  
dtypes: float64(12), int64(1)
memory usage: 545.2 KB

--- 最終的なデータフレーム (最初の5行) ---


,open,high,low,close,volume,turnover,open_norm,high_norm,low_norm,close_norm,SMA_14,RSI_14,target
timestamp,,,,,,,,,,,,,
2025-04-02 22:10:00+00:00,83889.9,83927.9,83508.6,83641.0,2106.624,1.763106e+08,1.002976,1.003430,0.998417,1.0,84444.292857,23.408539,0
2025-04-02 22:15:00+00:00,83641.0,83641.0,83257.7,83489.0,1271.640,1.060886e+08,1.001821,1.001821,0.997230,1.0,84359.021429,21.753413,1
2025-04-02 22:20:00+00:00,83489.0,83548.2,83237.2,83548.2,1004.540,8.379219e+07,0.999291,1.000000,0.996278,1.0,84291.528571,24.007095,0
2025-04-02 22:25:00+00:00,83548.2,83589.6,83373.2,83429.7,385.891,3.221438e+07,1.001420,1.001917,0.999323,1.0,84223.207143,22.603675,0
2025-04-02 22:30:00+00:00,83429.7,83429.8,83056.1,83082.7,1330.437,1.106877e+08,1.004177,1.004178,0.999680,1.0,84110.185714,19.085295,0


## 9. データの分割 (学習データとテストデータ)

機械学習モデルの性能を公平に評価するため、データを学習用とテスト用に分割します。
時系列データなので、過去のデータを学習に、未来のデータをテストに使用します。
例えば、全体の80%を学習データ、残りの20%をテストデータとします。

In [10]:
from sklearn.model_selection import train_test_split

if 'df_processed' in locals() and not df_processed.empty:
    # 特徴量として使用する列を指定
    # ドキュメントに基づき、正規化された価格情報とテクニカル指標を使用
    # 元のOHLCVやvolume, turnoverは含めない例
    features = ['open_norm', 'high_norm', 'low_norm', 'close_norm', 'SMA_14', 'RSI_14']
    # (他の特徴量を追加・削除する場合はこのリストを編集)

    X = df_processed[features]
    y = df_processed['target']

    # データの分割 (時系列を考慮するため shuffle=False を指定)
    test_size = 0.2 # テストデータの割合 (例: 20%)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=False)

    # 分割後のデータ件数を確認
    print("--- データ分割結果 ---")
    print(f"全データ数: {len(X)}")
    print(f"学習データ数 (X_train): {len(X_train)}")
    print(f"テストデータ数 (X_test): {len(X_test)}")
    print(f"学習データ期間: {X_train.index.min()} ~ {X_train.index.max()}")
    print(f"テストデータ期間: {X_test.index.min()} ~ {X_test.index.max()}")

    # 特徴量データ (X_train) の最初の数行を確認
    print("\n--- 学習データの特徴量 (X_train) の最初の5行 ---")
    display(X_train.head())

else:
    print("df_processed が存在しないか空です。前のステップを先に実行してください。")


--- データ分割結果 ---
全データ数: 4985
学習データ数 (X_train): 3988
テストデータ数 (X_test): 997
学習データ期間: 2025-04-02 22:10:00+00:00 ~ 2025-04-16 18:25:00+00:00
テストデータ期間: 2025-04-16 18:30:00+00:00 ~ 2025-04-20 05:30:00+00:00

--- 学習データの特徴量 (X_train) の最初の5行 ---


,open_norm,high_norm,low_norm,close_norm,SMA_14,RSI_14
timestamp,,,,,,
2025-04-02 22:10:00+00:00,1.002976,1.003430,0.998417,1.0,84444.292857,23.408539
2025-04-02 22:15:00+00:00,1.001821,1.001821,0.997230,1.0,84359.021429,21.753413
2025-04-02 22:20:00+00:00,0.999291,1.000000,0.996278,1.0,84291.528571,24.007095
2025-04-02 22:25:00+00:00,1.001420,1.001917,0.999323,1.0,84223.207143,22.603675
2025-04-02 22:30:00+00:00,1.004177,1.004178,0.999680,1.0,84110.185714,19.085295


## 10. モデルの学習・評価・バックテスト

分割したデータを使って、機械学習モデルを学習させ、テストデータで性能を評価します。
さらに、その予測結果を用いてバックテスト（取引シミュレーション）を行います。
ここではまず、シンプルな **Logistic Regression** モデルを使用します。

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import pandas as pd
import numpy as np

if 'X_train' in locals() and 'y_train' in locals() and 'X_test' in locals() and 'y_test' in locals():

    # --- 1. モデルの学習 ---
    print("--- 1. モデル学習開始 (Logistic Regression) ---")
    model = LogisticRegression(random_state=42) # 再現性のため random_state を指定
    model.fit(X_train, y_train)
    print("モデル学習完了。")

    # --- 2. モデル評価 (テストデータ) ---
    print("\n--- 2. モデル評価 (テストデータ) ---")
    # 予測ラベルの取得
    y_pred = model.predict(X_test)
    # 予測確率の取得 [クラス0の確率, クラス1(High)の確率]
    y_pred_proba = model.predict_proba(X_test)[:, 1] # クラス1 (High) の確率を取得

    # 評価指標の計算と表示
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy (正解率): {accuracy:.4f}")

    print("\nClassification Report:")
    # target_names=['Low (0)', 'High (1)'] を指定すると見やすい
    print(classification_report(y_test, y_pred, target_names=['Low (0)', 'High (1)']))

    print("\nConfusion Matrix:")
    # DataFrameで見やすく表示
    conf_matrix = confusion_matrix(y_test, y_pred)
    conf_matrix_df = pd.DataFrame(conf_matrix, index=['Actual Low (0)', 'Actual High (1)'], columns=['Predicted Low (0)', 'Predicted High (1)'])
    display(conf_matrix_df)

    # AUCスコア
    try:
        auc = roc_auc_score(y_test, y_pred_proba)
        print(f"\nAUC Score: {auc:.4f}")
    except ValueError as e:
        print(f"\nAUC Score計算エラー: {e}") # データに一方のクラスしか含まれない場合など

    # --- 3. バックテスト ---
    print("\n--- 3. バックテスト (シミュレーション) ---")

    # バックテスト用パラメータ設定
    payout_rate = 1.80 # ペイアウト率 (例: ハイロー1.8倍)
    required_win_rate = 1 / payout_rate # 損益分岐勝率 = 約0.555...
    # ドキュメントの考え方に基づき、損益分岐勝率を基準にエントリー閾値を設定
    # High確率 > 損益分岐勝率 で Highエントリー
    # Low確率 > 損益分岐勝率 (つまり High確率 < 1 - 損益分岐勝率) で Lowエントリー
    high_threshold = required_win_rate # Highエントリーする確率閾値 (これを超えたら)
    low_threshold = 1.0 - required_win_rate     # Lowエントリーする確率閾値 (これより低かったら)

    print(f"使用するペイアウト率: {payout_rate}")
    print(f"損益分岐勝率: {required_win_rate:.4f}")
    print(f"Highエントリー閾値 (High確率 >): {high_threshold:.4f}")
    print(f"Lowエントリー閾値 (High確率 <): {low_threshold:.4f}")


    # バックテスト用DataFrameを作成
    df_backtest = pd.DataFrame({
        'actual': y_test,                # 実際の High/Low (0 or 1)
        'predict_proba_high': y_pred_proba # High になる予測確率
    }, index=X_test.index)

    # エントリーシグナルを生成
    # High確率が high_threshold を超えたら High (1) エントリー
    # High確率が low_threshold を下回ったら Low (0) エントリー
    # それ以外は Wait (-1) エントリーしない
    conditions = [
        df_backtest['predict_proba_high'] > high_threshold,
        df_backtest['predict_proba_high'] < low_threshold
    ]
    choices = [1, 0] # Highエントリーシグナル=1, Lowエントリーシグナル=0
    df_backtest['signal'] = np.select(conditions, choices, default=-1)

    # エントリーした取引のみを抽出
    df_trades = df_backtest[df_backtest['signal'] != -1].copy()

    # 勝敗を判定 (予測シグナルと実際の値が一致したら勝ち=1, 負け=0)
    df_trades['win'] = np.where(df_trades['signal'] == df_trades['actual'], 1, 0)

    # 損益を計算 (勝ち: payout_rate - 1, 負け: -1)
    profit_per_win = payout_rate - 1.0
    loss_per_lose = -1.0
    df_trades['profit'] = np.where(df_trades['win'] == 1, profit_per_win, loss_per_lose)

    # バックテスト結果の集計
    total_trades = len(df_trades)
    total_wins = df_trades['win'].sum()
    total_profit = df_trades['profit'].sum()

    if total_trades > 0:
        win_rate = total_wins / total_trades
        average_profit = total_profit / total_trades
        # プロフィットファクター (総利益 / 総損失)
        total_gross_profit = df_trades[df_trades['profit'] > 0]['profit'].sum()
        total_gross_loss = abs(df_trades[df_trades['profit'] < 0]['profit'].sum())
        # total_gross_loss が 0 の場合の処理を追加
        profit_factor = total_gross_profit / total_gross_loss if total_gross_loss > 0 else np.inf if total_gross_profit > 0 else 0 # 損失ゼロなら無限大(利益あり) or ゼロ(利益なし)

        print("\n--- バックテスト結果 ---")
        print(f"テストデータ期間: {df_trades.index.min()} ~ {df_trades.index.max()}")
        print(f"総取引回数 (シグナル発生): {total_trades}")
        print(f"勝利数: {total_wins}")
        print(f"勝率: {win_rate:.4f}")
        print(f"総損益 (1単位あたり): {total_profit:.4f}")
        print(f"平均損益 (1取引あたり): {average_profit:.4f}")
        print(f"プロフィットファクター: {profit_factor:.4f}")
    else:
        print("\n--- バックテスト結果 ---")
        print("テスト期間中にエントリーシグナルが発生しませんでした。")
        print(f"参考: テストデータ期間 {X_test.index.min()} ~ {X_test.index.max()}")
        print(f"参考: High予測確率の最大値: {df_backtest['predict_proba_high'].max():.4f}")
        print(f"参考: High予測確率の最小値: {df_backtest['predict_proba_high'].min():.4f}")


else:
    print("学習データまたはテストデータが存在しません。前のステップを先に実行してください。")

--- 1. モデル学習開始 (Logistic Regression) ---
モデル学習完了。

--- 2. モデル評価 (テストデータ) ---
Accuracy (正解率): 0.5226

Classification Report:
              precision    recall  f1-score   support

     Low (0)       0.51      0.60      0.55       490
    High (1)       0.54      0.45      0.49       507

    accuracy                           0.52       997
   macro avg       0.52      0.52      0.52       997
weighted avg       0.52      0.52      0.52       997


Confusion Matrix:


,Predicted Low (0),Predicted High (1)
Actual Low (0),294,196
Actual High (1),280,227



AUC Score: 0.5266

--- 3. バックテスト (シミュレーション) ---
使用するペイアウト率: 1.8
損益分岐勝率: 0.5556
Highエントリー閾値 (High確率 >): 0.5556
Lowエントリー閾値 (High確率 <): 0.4444

--- バックテスト結果 ---
テスト期間中にエントリーシグナルが発生しませんでした。
参考: テストデータ期間 2025-04-16 18:30:00+00:00 ~ 2025-04-20 05:30:00+00:00
参考: High予測確率の最大値: 0.5438
参考: High予測確率の最小値: 0.4548
